# 文本分类与打标签

本章内容

1. 本章内容概述
2. 基于卷积神经网络的文本分类模型详解
3. 基于RNN/LSTM的文本分类模型详解
4. 构建卷积神经网络完成影评褒贬分类
5. 构建LSTM模型完成新闻文本分类

## 本章内容概述

文本分类是一个常见的人工智能任务。通常我们需要对

## 基于卷积神经网络的文本分类模型

在这一节里，我们将介绍如何使用卷积神经网络（CNN）模型进行文本分类。

- 我们会首先介绍卷积神经网络的原理及相关的概念，比如池化等。
- 其次我们介绍一维卷积网络在文字上的应用。
- 随后，通过与图像数据的对比引入如何将二维卷积神经网络方法运用到文本信息建模中。
- 最后，我们将拓展单通道二维卷积神经网络到多通道卷积神经网络模型。

### 什么是卷积神经网络
卷积神经网络模型是利用卷积滤子对输入数据进行分析的技术。在数学里，卷积一词的含义是指通过两个函数f和g生成第三个函数h的操作，产出函数f经过函数g变换后的形态。通常函数g会对原始函数进行平移或者翻转操作。卷积既指代这个操作，也指代这个操作的结果函数。卷积还可以被看作是“加权移动平均”的推广。 我们利用下面这个wiki的图来演示卷积操作。
<img src="./pics/Convolution3.png" width="450">
这个图中有五行图片。
- 第一行的两幅图分别是原始函数f(t)和g(t)。假设两个函数都用 ${\tau }$ 来表示，从而得到f( $\tau$ )和g( $ \tau$ )。
- 第二行的图片希纳是将函数g($\tau$)向右移动t个单位并翻转后得到函数g( $\tau -t$)的图像。将g( $\tau -t$)翻转至纵轴另一侧，得到g(-($\tau$  -t))即g(t- $\tau$ )的图像。
- 第三行的图片显示了当时间变量$\tau$取不同值时，函数g(t-$\tau$ ) 沿着时间轴$\tau$“滑动”的动作。
- 第四行的图片显示了当f(t)和g(t)两函数交会时,两函数乘积的积分值。这个积分值可以理解为使用g(t-$\tau$)做为加权函数，来对f($\tau$)取加权值的结果。
- 第五行的图片显示了这个加权平均的持续过程。

最后得到的加权平均的结果就是f和g的卷积。如果f（t）是一个单位脉冲，我们得到的乘积就是g（t）本身，称为冲激响应。

<img src="./pics/Convolution_self.gif" width="450">
以上来自wiki的图显示了两个方形脉冲波的卷积。其中
函数"g"首先对 $\tau =0$反射，接着平移"t"，成为 $ g(t-\tau )$。那么重叠部分的面积就相当于"t"处的卷积，其中横坐标代表待变量 $ \tau $以及新函数 $ f\ast g$的自变量"t"。

<img src="./pics/Convolution_spiky.gif" width="450">
以上来自wiki的图显示了方形脉冲波和指数衰退的脉冲波的卷积，同样地重叠部分面积就相当于"t"处的卷积。注意到因为"g"是对称的，所以在这两张图中，反射并不会改变它的形状。

以上是对一维卷积的解释。卷积神经网络最常见于图像识别中，因此通常是使用二维卷积。下面简要说明（图片来源：http:www.wildml.com）。
<img src="./pics/CNNconcept.png" width="500">

二维卷积网络通常使用在与图像相关的建模工作中。在一个二维卷积网络里面，滤子是一个小的矩阵，比如一个$2\times 2$或者$3 \times 3$的矩阵。这个小的滤子对于输入的代表图像的矩阵的相应局部进行元素级别的乘积与求和，输出的矩阵即是二维卷积的结果。我们使用下图来解释。

<img src="./pics/Convolution_operation.png" width="500">

在上图中，左边较大的矩阵是原始输入图像的矩阵，中间蓝色边框的小矩阵代表滤子。在这个例子中，左边矩阵红框的局部与蓝色边框的滤子进行对应位置的元素级别的乘积操作：
$$
1 \times 1 + 0 \times 0 + 0 \times 1  + 1\times 0 + 1\times 1 + 0\times 0 + 1\times 1 + 1\times 0 + 1\times 1 = 4
$$
得到右边矩阵绿色框内的数据。

与上面提到的一维空间的例子类似，滤子会在原始矩阵内按照一定规律移动，每移动到一个新的位置就与该位置的相应元素进行类似的操作，得到卷积后的结果。过滤器以多大的跨度上下或左右平移地扫描移动的规律叫做“步长”（Stride）。在上例中，步长为1，即每完成一次卷积的操作后，滤子往右或者往下移动一个格子的距离，再进行卷积操作，直到移动到边界为止。当然，步长也可以为别的数字。卷积可以有效降低输入数据的维度，在保留核心信息的时候突出特定特征。

我们看到，一个滤子通过数值的排列，可以突出某种特定数据模式，从而帮助模型寻找所需要的特征。以下面的例子为例。左边的图展示了一个曲线特征。这个特征可以对应于各类图片的外凸曲线，比如车轮的一部分轮廓，数字0的一部分，等等，本身是一个抽象的特征。右边的图是对应于该特征的滤子矩阵，中间的图是输入的图。当滤子掠过这幅图片的区域，卷积计算会得到一个很大的数值输出，从而凸显图片里具备这个曲线轮廓特征。
<img src="./pics/CNNcurve.png" width="500">

滤子在图像领域应用历史悠久，有很多经典的滤子，通过不同的数值组合，可以产生不同的过滤效果。下图展示了三种典型的图像处理领域的滤子。
<img src="./pics/Convolution_filters.png" width="500">

在卷积神经网络模型中，滤子不是预先定义好的，而是作为参数在模型训练中优化得到的，因此相比预先定义好的滤子，卷积神经网络模型能够抓取更有效的图像特征。这些特征通常反映了抽象的概念，比如一个物体的边界，或者两个物体之间的距离，等。通过叠加卷积层，可以实现不停抽象的过程。比如在人脸侦测的任务中，第一层卷积神经网络可以将人脸的边界抽象出来，而第二层卷积神经网络可以将人眼之间的距离抽象出来，从而实现层级的特征抽象。这点非常类似于人脑的工作原理。

除了卷积滤子和步长之外，卷积神经网络里面还有一个概念叫“池化”（Pooling）。池化层将给定的特征图像中的几个空间上相邻的数据通过特定的函数输出，能显著降低模型参数的数量。池化有时也被称为subsampling或者downsampling。池化能在降低参数数量的同时尽可能地保留重要信息。池化对应的函数通常有三种：

- 最大 ：MaxPooling，取相邻格子中最大的值
- 平均 ：Average Pooling，取相邻格子中所有值的平均值
- 求和 ： Sum Pooling，取相邻格子中所有制的总和

下图显示了采用最大池化方法的结果，因为采用了$2\times 2$矩阵，步长为2的池化过程，因此池化后的结果为$2\times 2$的矩阵，保留了每个区域最大的数值。
<img src="./pics/MaxPooling2.png" width="500">


对于经由过滤器局部扫描后的卷积层图像，由于处理边界不同，一般有两种方式，对应于“补齐”（padding）这个概念。补齐有两种方法，一种方法是在局部扫描过程中对图像边界以外的一层或多层填上0，平移的时候可以移出边界外到达0的区域。这样的好处是在以1 为步长的局部扫描完以后，所得的新图像和原图像长宽一致，被称作zero padding(same padding)。另一种是不对边界外做任何0 的假定，所有平移都在边界内，被称作valid padding，使用这种方式通常扫描完的图像尺寸会比原来的小。



### 使用一维卷积神经网络进行文本分类模型

我们从前面了解到，卷积神经网络具备优异的特征萃取能力，但是通常用在图像数据的建模中，如何用在文本数据中呢？下面我们就来解开谜底。

在实际使用中，一句话都是以单词索引下标表示的列表，长短不一，因此先要对其进行补齐，将每一句话补齐为同样长度，才方便使用。这时可以选择的补齐长度不一，既可以选择最大的句子长度作为需要补齐的长度，也可以选择所有句子长度的中位值来补齐。

把句子补齐之后，每一句话就称为一个整数的时间序列，是一串连续的信号。在使用卷积层的时机上，可以有两种选择。一是直接在这个时候使用，将卷积操作应用于连续的单词下标列表，直接抽取特征；二是先将补齐后的每一句话通过嵌入层映射到一个稍微低维度的致密空间，再对该致密空间的连续信号使用卷积算子，抽取特征。第一种方法因为单词的下标顺序并无特定含义，因此效果不一定好。

在keras中，我们可以使用Embedding层来引入嵌入层，通常直接用在输入层后。我们使用Conv1D来引入一维卷积层。我们甚至可以对信号进行多次卷积。下面分别介绍这两者用法。



### 使用二维卷积神经网络进行文本分类模型

我们也可以使用二维卷积神经网络对文本建模。这是因为，我们可以将一个文本中相邻的单词对应的词向量叠加起来，那么对于任意给定的窗口，这些叠加的词向量就构成一个类似图片的二维“面板”数据。

从《词嵌入与词向量》这节课中我们了解到可以将一组文本中的任意单词映射到一个指定维度的新的空间。这个指定的维度通常比文本中全部单词所需要的维度低很多，但是通常仍然需要数百维。我们需要将使用卷积神经网络将这数百维的向量再次压缩到更低的向量中。对于这样一个一维的连续信号，完全可以使用上面提到的一维卷积方法，一方面可以将维度从数百维降为更低维度，另一方面又能尽可能萃取最有用的信号。

以下列文本为例（已经分词）。

## 基于RNN/LSTM的文本分类模型

In [1]:
import pandas as pd
import numpy as np
import cntext 
import jieba

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file = './nlp_data/douban_moview_reviews/DMSC.csv'
raw = pd.read_csv(file, header=0)
raw.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


In [3]:
%%time
documents = [raw.Comment[i] for i in range(raw.shape[0]) ]

Wall time: 55.5 s


In [4]:
tokenizer = cntext.cnTokenizer()
%time tokenizer.fit_on_cntexts(documents)
V = len(tokenizer.word_index) + 1
%time text_sequences=tokenizer.cntexts_to_sequences(documents)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xieliang\AppData\Local\Temp\jieba.cache
Loading model cost 0.667 seconds.
Prefix dict has been built succesfully.


Wall time: 7min 34s
Wall time: 7min 18s


In [20]:
temp = list(text_sequences[:2])
print(len(temp))
print(temp)
text_sequences_padded = sequence.pad_sequences(temp, maxlen=400)

2
[[1, 504, 1690, 9, 89, 4580, 67, 43, 354, 1], [1, 131, 201, 1, 392, 156, 19378, 1, 1075, 19, 36, 1111, 228, 47, 430, 1, 352, 305, 2, 169, 9, 1615, 4053, 1, 16587, 612, 55, 1066, 5226, 9, 334, 4679, 1, 4491, 1, 4476, 1, 4491, 2, 14073, 73, 522, 30, 107, 10, 47, 3634, 14698, 703, 2, 700, 1573, 683, 321, 787, 1, 30, 392, 9269, 131, 2127, 55, 8680, 1, 221, 69, 3368, 963, 11212, 1, 3752, 2, 14156, 2157, 1, 223, 50, 29025, 65, 69, 3294, 1]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    

In [24]:
 y = raw.Star

In [6]:
from keras.datasets import imdb
max_features=5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [12]:
print(x_train[:2])
print(y_train[:2])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     1   14   22   16   43  530  973 1622 1385   65  458 4468   

In [34]:

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras.utils import to_categorical

In [44]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
#x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
%time text_sequences_padded = sequence.pad_sequences(text_sequences, maxlen=maxlen)
y_binary = to_categorical(y)

cutoff = 1500000
x_train = text_sequences_padded[:cutoff]
x_test = text_sequences_padded[cutoff:]

y_train = y_binary[:cutoff]
y_test = y_binary[cutoff:]

Wall time: 12.5 s


In [45]:
len(text_sequences_padded)
y_binary.shape

(2125056, 6)

In [46]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 400, 50)           250000    
_________________________________________________________________
dropout_19 (Dropout)         (None, 400, 50)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 250)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 250)               62750     
_________________________________________________________________
dropout_20 (Dropout)         (None, 250)               0         
_________________________________________________________________
activation_19 (Activation)   (None, 250)               0         
__________

In [47]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 1500000 samples, validate on 625056 samples
Epoch 1/2
1500000/1500000 [==============================] - 337s 225us/step - loss: 1.9296 - acc: 0.1192 - val_loss: 2.1972 - val_acc: 0.1075
Epoch 2/2
1500000/1500000 [==============================] - 336s 224us/step - loss: 1.7109 - acc: 0.0737 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
